# Full audio

## Feature

### Set up

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
from IPython.display import clear_output, Audio
import IPython.display as ipd
import os
import numpy as np
import shutil
import pandas as pd
import logging
import warnings

warnings.filterwarnings("ignore")


logger = logging.getLogger('COUGHVID - HYBRID MODEL FULL AUDIO')
logger.setLevel(logging.DEBUG)

# create console handler and set level to debug
ch = logging.StreamHandler()
ch.setLevel(logging.DEBUG)

# create formatter
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')

# add formatter to ch
ch.setFormatter(formatter)

# add ch to logger
logger.addHandler(ch)
pd.set_option('display.max_colwidth', None)
clear_output()

In [ ]:
n_mfcc = 26

root = '/content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet'

meta_coswara = os.path.join(root, 'full_audio/coswara_metadata_for_label_matching.csv')
meta_coughvid = os.path.join(root, 'full_audio/coughvid_metadata_for_label_matching.csv')


coswara_feature = os.path.join(root, f'full_audio/mfcc_{n_mfcc}_coswara_features.npy')
coughvid_feature = os.path.join(root, f'full_audio/mfcc_{n_mfcc}_coughvid_features.npy')

### Feature preparation

#### Metadata

In [ ]:
# process metadata
meta_df = pd.read_csv(meta_coughvid)
meta_df.head()

,uuid,datetime,cough_detected,latitude,longitude,age,gender,respiratory_condition,fever_muscle_pain,status,...,stridor_3,choking_3,congestion_3,nothing_3,cough_type_3,diagnosis_3,severity_3,cough_path,label,error
0,00039425-7f3a-42aa-ac13-834aaa2b6b92,2020-04-13T21:30:59.801831+00:00,0.9609,31.3,34.8,15.0,male,False,False,healthy,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,/content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/data/coughvid/coughvid_cough_data/00039425-7f3a-42aa-ac13-834aaa2b6b92.webm,healthy,0
1,0009eb28-d8be-4dc1-92bb-907e53bc5c7a,2020-04-12T04:02:18.159383+00:00,0.9301,40.0,-75.1,34.0,male,True,False,healthy,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,/content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/data/coughvid/coughvid_cough_data/0009eb28-d8be-4dc1-92bb-907e53bc5c7a.webm,healthy,0
2,0012c608-33d0-4ef7-bde3-75a0b1a0024e,2020-04-15T01:03:59.029326+00:00,0.0482,-16.5,-71.5,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,/content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/data/coughvid/coughvid_cough_data/0012c608-33d0-4ef7-bde3-75a0b1a0024e.webm,NaN,0
3,001328dc-ea5d-4847-9ccf-c5aa2a3f2d0f,2020-04-13T22:23:06.997578+00:00,0.9968,NaN,NaN,21.0,male,False,False,healthy,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,/content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/data/coughvid/coughvid_cough_data/001328dc-ea5d-4847-9ccf-c5aa2a3f2d0f.webm,healthy,0
4,001c85a8-cc4d-4921-9297-848be52d4715,2020-04-17T15:24:35.822355+00:00,0.0735,40.6,-3.6,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,/content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/data/coughvid/coughvid_cough_data/001c85a8-cc4d-4921-9297-848be52d4715.webm,NaN,0


In [ ]:
logger.info(meta_df.shape)

2022-04-30 15:21:40,681 - COUGHVID - HYBRID MODEL FULL AUDIO - INFO - (20072, 43)


In [ ]:
mfcc_feature = np.load(coughvid_feature)
meta_df = meta_df[meta_df['error'] == 0].reset_index(drop=True)

In [ ]:
mfcc_feature[0].shape, len(mfcc_feature), meta_df.shape

((234, 246), 20072, (20072, 43))

In [ ]:
DROP_COLS = [
             'uuid',
             'error',
             'datetime',
             'cough_path',
             'cough_detected',
             ]

meta_df.isna().sum()

uuid                         0
datetime                     0
cough_detected               0
latitude                  8326
longitude                 8326
age                       9604
gender                    8758
respiratory_condition     8758
fever_muscle_pain         8758
status                    8758
quality_1                19252
cough_type_1             19252
dyspnea_1                19252
wheezing_1               19252
stridor_1                19252
choking_1                19252
congestion_1             19252
nothing_1                19252
diagnosis_1              19252
severity_1               19252
quality_2                19252
cough_type_2             19253
dyspnea_2                19252
wheezing_2               19252
stridor_2                19252
choking_2                19252
congestion_2             19252
nothing_2                19252
diagnosis_2              19252
severity_2               19252
quality_3                19252
dyspnea_3                19252
wheezing

In [ ]:
meta_df.dtypes

uuid                      object
datetime                  object
cough_detected           float64
latitude                 float64
longitude                float64
age                      float64
gender                    object
respiratory_condition     object
fever_muscle_pain         object
status                    object
quality_1                 object
cough_type_1              object
dyspnea_1                 object
wheezing_1                object
stridor_1                 object
choking_1                 object
congestion_1              object
nothing_1                 object
diagnosis_1               object
severity_1                object
quality_2                 object
cough_type_2              object
dyspnea_2                 object
wheezing_2                object
stridor_2                 object
choking_2                 object
congestion_2              object
nothing_2                 object
diagnosis_2               object
severity_2                object
quality_3 

In [ ]:
# fill na with unknown value
# meta_df[['latitude', 'longitude', 'age']] = meta_df[['latitude', 'longitude', 'age']].fillna(99999)
meta_df.fillna('unknown', inplace=True)
meta_df.isna().sum()

uuid                     0
datetime                 0
cough_detected           0
latitude                 0
longitude                0
age                      0
gender                   0
respiratory_condition    0
fever_muscle_pain        0
status                   0
quality_1                0
cough_type_1             0
dyspnea_1                0
wheezing_1               0
stridor_1                0
choking_1                0
congestion_1             0
nothing_1                0
diagnosis_1              0
severity_1               0
quality_2                0
cough_type_2             0
dyspnea_2                0
wheezing_2               0
stridor_2                0
choking_2                0
congestion_2             0
nothing_2                0
diagnosis_2              0
severity_2               0
quality_3                0
dyspnea_3                0
wheezing_3               0
stridor_3                0
choking_3                0
congestion_3             0
nothing_3                0
c

In [ ]:
# drop cols
meta_df.drop(DROP_COLS, axis=1, inplace=True)

In [ ]:
# take labels
meta_df.label = meta_df.label.map({
    'healthy': 0,
    'symptomatic': 1,
    'covid-19': 1
})
logger.info(meta_df.label.value_counts().to_dict())

2022-04-30 15:21:50,307 - COUGHVID - HYBRID MODEL FULL AUDIO - INFO - {0.0: 8562, 1.0: 1742}


In [ ]:
meta_df.label.value_counts().sum()

10304

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

for col in meta_df.drop(['label'], axis=1).columns:
  meta_df[col] = meta_df[col].astype('string')
  meta_df[col] = le.fit_transform(meta_df[col].values)

In [ ]:
meta_feature = np.array(meta_df.drop(['label'], axis=1))

#### Features

##### 2d

In [ ]:
label_0 = meta_df[meta_df['label']==0].copy().reset_index(drop=True)
label_1 = meta_df[meta_df['label']==1].copy().reset_index(drop=True)

In [ ]:
# get index of each label 0, 1 to match with extracted features
index_1 = meta_df[meta_df['label']==1].index.tolist()
index_0 = meta_df[meta_df['label']==0].index.tolist()
logger.info(f'{len(index_1)}, {len(index_0)}')

2022-04-30 15:21:51,245 - COUGHVID - HYBRID MODEL FULL AUDIO - INFO - 1742, 8562


In [ ]:
# get features for each class
feature_0 = mfcc_feature[index_0]
feature_1 = mfcc_feature[index_1]

In [ ]:
print(len(feature_1))

1742


In [ ]:
del mfcc_feature

In [ ]:
# split data and reshape feature
def data_split(feature_0, feature_1, label0, label1):
  train = np.concatenate((feature_0[:6850], feature_1[:1394]), axis=0)
  val = np.concatenate((feature_0[6850:7706], feature_1[1394:1568]), axis=0)
  test = np.concatenate((feature_0[7706:], feature_1[1568:]), axis=0)

  y_train = pd.concat([label0[:6850], label1[:1394]], ignore_index=True)
  y_val = pd.concat([label0[6850:7706], label1[1394:1568]], ignore_index=True)
  y_test = pd.concat([label0[7706:], label1[1568:]], ignore_index=True)

  return train, val, test,\
         np.array(y_train.label), np.array(y_val.label), np.array(y_test.label)

In [ ]:
X_train_2d, X_val_2d, X_test_2d, y_train_2d, y_val_2d, y_test_2d = data_split(feature_0, feature_1, label_0, label_1)
print(X_train_2d.shape)

(8244, 234, 246)


In [ ]:
y_train_2d

array([0., 0., 0., ..., 1., 1., 1.])

In [ ]:
del feature_0
del feature_1

##### 1d

In [ ]:
feature_0 = meta_feature[index_0]
feature_1 = meta_feature[index_1]

In [ ]:
X_train_1d, X_val_1d, X_test_1d, y_train_1d, y_val_1d, y_test_1d = data_split(feature_0, feature_1, label_0, label_1)

In [ ]:
from sklearn.utils import shuffle

X_train_2d, X_train_1d, y_train_2d, y_train_1d = shuffle(X_train_2d, X_train_1d, y_train_2d, y_train_1d, random_state=42)
X_val_2d, X_val_1d, y_val_2d, y_val_1d = shuffle(X_val_2d, X_val_1d, y_val_2d, y_val_1d, random_state=42)
X_test_2d, X_test_1d, y_test_2d, y_test_1d = shuffle(X_test_2d, X_test_1d, y_test_2d, y_test_1d, random_state=42)

## Model

In [ ]:
X_train_2d, X_val_2d, X_test_2d = X_train_2d[..., np.newaxis], X_val_2d[..., np.newaxis], X_test_2d[..., np.newaxis]
X_train_2d.shape

(8244, 234, 246, 1)

In [ ]:
X_train_1d, X_val_1d, X_test_1d = X_train_1d[..., np.newaxis], X_val_1d[..., np.newaxis], X_test_1d[..., np.newaxis]
X_train_1d.shape

(8244, 37, 1)

In [ ]:
X_train_1d.shape

(8244, 37, 1)

In [ ]:
import tensorflow as tf

y_train_2d = tf.one_hot(y_train_2d, depth=2)
y_test_2d = tf.one_hot(y_test_2d, depth=2)
y_val_2d = tf.one_hot(y_val_2d, depth=2)

In [ ]:
SAVED_MODEL_PATH = os.path.join(root, 'saved_model')
EPOCHS = 40
PATIENCE = 5
LEARNING_RATE = 0.001

In [ ]:
from tensorflow.keras.layers import Conv1D, Conv2D, MaxPooling1D, MaxPooling2D, Concatenate
from datetime import datetime

from tensorflow.keras.layers import Input, DepthwiseConv2D
from tensorflow.keras.layers import Conv2D, BatchNormalization
from tensorflow.keras.layers import ReLU, Activation, AvgPool2D, Flatten, Dense
from tensorflow.keras.layers import Dropout, LSTM, GRU, Bidirectional

from tensorflow.keras import Model



# MobileNet block

def mobilnet_block (x, filters, strides):
    
    x = DepthwiseConv2D(kernel_size = 3, strides = strides, padding = 'same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    x = Conv2D(filters = filters, kernel_size = 1, strides = 1)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    return x

def hybrid_model(input_1, input_2):

  input_1 = Input(input_1[1:])

  x = Conv2D(filters = 32, kernel_size = 1, strides = 2, padding = 'same')(input_1)
  x = BatchNormalization()(x)
  x = ReLU()(x)

  # main part of the model

  x = mobilnet_block(x, filters = 64, strides = 1)
  x = mobilnet_block(x, filters = 128, strides = 2)
  x = mobilnet_block(x, filters = 128, strides = 1)
  x = mobilnet_block(x, filters = 256, strides = 2)
  x = mobilnet_block(x, filters = 256, strides = 1)
  x = mobilnet_block(x, filters = 512, strides = 2)

  for _ in range (5):
      x = mobilnet_block(x, filters = 512, strides = 1)
  x = mobilnet_block(x, filters = 1024, strides = 2)
  x = mobilnet_block(x, filters = 1024, strides = 1)
  x = AvgPool2D (pool_size = 7, strides = 1, data_format='channels_first')(x)

  x = Flatten()(x)
  x = Dense(units=500, activation='relu')(x)
  x = Dense(units=256, activation='relu')(x)
  x = Dense(units=128, activation='relu')(x)
  x = Dense(units=64, activation='relu')(x)
  # add meta
  # input_2 = Input((7380, 128000))
  input_2 = Input(input_2[1:])

  y = Conv1D(filters=64, kernel_size=(3), strides=1, padding='same')(input_2)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4)(y)
  y = Dropout(0.5)(y)

  # LFLB2
  y = Conv1D(filters=64, kernel_size=3, strides=1, padding='same')(y)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4, padding='same')(y)
  y = Dropout(0.25)(y)

  # LFLB3
  y = Conv1D(filters=128, kernel_size=3, strides=1, padding='same')(y)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4, padding='same')(y)
  y = Dropout(0.25)(y)

  # LFLB4
  y = Conv1D(filters=128, kernel_size=3, strides=1, padding='same')(y)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4, padding='same')(y)
  y = Dropout(0.25)(y)

  y = LSTM(units = 128, return_sequences = True, name='lstm_layer')(y)
  y = Dropout(rate = 0.2)(y) 
  y = BatchNormalization()(y)
  y = Dropout(rate = 0.2)(y)
  y = Flatten()(y)
  y = Dense(64)(y)


  merged = Concatenate()([x, y])
  output = Dense(2, activation='softmax')(merged)

  merged_model = Model(inputs=[input_1, input_2], outputs=[output])

  return merged_model

input_2d = X_train_2d.shape
input_1d = X_train_1d.shape
model = hybrid_model(input_2d, input_1d)

In [ ]:
opt = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)
auc = tf.keras.metrics.AUC()
loss = tf.keras.losses.BinaryCrossentropy()

model.compile(optimizer=opt,
              loss=loss,
              metrics=[auc])

In [ ]:
fp = './bestweight.h5'

check = tf.keras.callbacks.ModelCheckpoint(fp,
                                           monitor='val_auc',
                                           save_best_only=True,
                                           mode='max')

earlystop_callback = tf.keras.callbacks.EarlyStopping(monitor="val_auc",
                                                      min_delta=0.001,
                                                      mode='max',
                                                      patience=5)

log_dir = "logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [ ]:
for i in [8, 16, 32, 64, 128]:
  BATCH_SIZE = i

  history = model.fit([X_train_2d, X_train_1d],\
                      [y_train_2d, y_train_1d],\
                      epochs=EPOCHS,\
                      batch_size=BATCH_SIZE,\
                      validation_data=([X_val_2d, X_val_1d], [y_val_2d, y_val_1d]),\
                      callbacks=[check, earlystop_callback, tensorboard_callback],\
                      shuffle=True)
  
  model.evaluate([X_test_2d, X_test_1d], [y_test_2d, y_test_1d])

  model.save(os.path.join(SAVED_MODEL_PATH, f'full_audio_coughvid_mfcc_{n_mfcc}_hybrid_model_{EPOCHS}_{BATCH_SIZE}_{LEARNING_RATE}'))

  del model

  model = hybrid_model(input_2d, input_1d)

  model.compile(optimizer=opt,
               loss=loss,
               metrics=[auc])

Epoch 1/40
1031/1031 [==============================] - 83s 63ms/step - loss: 0.5020 - auc: 0.8375 - val_loss: 0.4682 - val_auc: 0.8269
Epoch 2/40
1031/1031 [==============================] - 64s 62ms/step - loss: 0.4668 - auc: 0.8408 - val_loss: 0.4587 - val_auc: 0.8309
Epoch 3/40
1031/1031 [==============================] - 63s 61ms/step - loss: 0.4610 - auc: 0.8404 - val_loss: 0.4929 - val_auc: 0.8273
Epoch 4/40
1031/1031 [==============================] - 64s 62ms/step - loss: 0.4607 - auc: 0.8392 - val_loss: 0.4815 - val_auc: 0.8287
Epoch 5/40
1031/1031 [==============================] - 64s 62ms/step - loss: 0.4552 - auc: 0.8453 - val_loss: 0.4580 - val_auc: 0.8273
Epoch 6/40
1031/1031 [==============================] - 64s 62ms/step - loss: 0.4580 - auc: 0.8385 - val_loss: 3.0355 - val_auc: 0.8315
Epoch 7/40
33/33 [==============================] - 1s 28ms/step - loss: 0.4559 - auc: 0.8396


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/full_audio_coughvid_mfcc_26_hybrid_model_40_8_0.001/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/full_audio_coughvid_mfcc_26_hybrid_model_40_8_0.001/assets


Epoch 1/40
516/516 [==============================] - 62s 107ms/step - loss: 0.5414 - auc: 0.8359 - val_loss: 0.4750 - val_auc: 0.8411
Epoch 2/40
516/516 [==============================] - 53s 103ms/step - loss: 0.4575 - auc: 0.8454 - val_loss: 0.4577 - val_auc: 0.8343
Epoch 3/40
516/516 [==============================] - 53s 103ms/step - loss: 0.4545 - auc: 0.8463 - val_loss: 0.4541 - val_auc: 0.8389
Epoch 4/40
516/516 [==============================] - 53s 104ms/step - loss: 0.4569 - auc: 0.8425 - val_loss: 0.4652 - val_auc: 0.8272
Epoch 5/40
516/516 [==============================] - 53s 104ms/step - loss: 0.4561 - auc: 0.8459 - val_loss: 0.4735 - val_auc: 0.8283
Epoch 6/40
33/33 [==============================] - 1s 28ms/step - loss: 0.4688 - auc: 0.8425


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/full_audio_coughvid_mfcc_26_hybrid_model_40_16_0.001/assets


...

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/full_audio_coughvid_mfcc_26_hybrid_model_40_64_0.001/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/full_audio_coughvid_mfcc_26_hybrid_model_40_64_0.001/assets


Epoch 1/40
65/65 [==============================] - 53s 711ms/step - loss: 0.9112 - auc: 0.8087 - val_loss: 0.5908 - val_auc: 0.8250
Epoch 2/40
65/65 [==============================] - 45s 700ms/step - loss: 0.4628 - auc: 0.8411 - val_loss: 0.4956 - val_auc: 0.8279
Epoch 3/40
65/65 [==============================] - 46s 701ms/step - loss: 0.4510 - auc: 0.8522 - val_loss: 0.5517 - val_auc: 0.8404
Epoch 4/40
65/65 [==============================] - 46s 704ms/step - loss: 0.4506 - auc: 0.8518 - val_loss: 0.5420 - val_auc: 0.8333
Epoch 5/40
65/65 [==============================] - 46s 702ms/step - loss: 0.4373 - auc: 0.8681 - val_loss: 0.5201 - val_auc: 0.8426
Epoch 6/40
65/65 [==============================] - 48s 747ms/step - loss: 0.4294 - auc: 0.8751 - val_loss: 0.5169 - val_auc: 0.8446
Epoch 7/40
65/65 [==============================] - 46s 712ms/step - loss: 0.4209 - auc: 0.8830 - val_loss: 0.4643 - val_auc: 0.8395
Epoch 8/40
65/65 [==============================] - 46s 712ms/step - 

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/full_audio_coughvid_mfcc_26_hybrid_model_40_128_0.001/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/full_audio_coughvid_mfcc_26_hybrid_model_40_128_0.001/assets


## Standard scaler model

In [ ]:
X_train_2d.shape

(8244, 234, 246)

In [ ]:
from sklearn.preprocessing import StandardScaler


sc = StandardScaler()

num_instances, first_dim, second_dim = X_train_2d.shape
X_train_2d = np.reshape(X_train_2d, newshape=(-1, second_dim))
X_train_2d = sc.fit_transform(X_train_2d)
X_train_2d = np.reshape(X_train_2d, newshape=(num_instances, first_dim, second_dim))


num_instances, first_dim, second_dim = X_val_2d.shape
X_val_2d = np.reshape(X_val_2d, newshape=(-1, second_dim))
X_val_2d = sc.transform(X_val_2d)
X_val_2d = np.reshape(X_val_2d, newshape=(num_instances, first_dim, second_dim))


num_instances, first_dim, second_dim = X_test_2d.shape
X_test_2d = np.reshape(X_test_2d, newshape=(-1, second_dim))
X_test_2d = sc.transform(X_test_2d)
X_test_2d = np.reshape(X_test_2d, newshape=(num_instances, first_dim, second_dim))

# reshape training data

X_train_2d, X_val_2d, X_test_2d = X_train_2d[..., np.newaxis], X_val_2d[..., np.newaxis], X_test_2d[..., np.newaxis]
X_train_2d.shape

(8244, 234, 246, 1)

In [ ]:
X_train_1d, X_val_1d, X_test_1d = X_train_1d[..., np.newaxis], X_val_1d[..., np.newaxis], X_test_1d[..., np.newaxis]
X_train_1d.shape

(8244, 37, 1)

In [ ]:
# reshape target
import tensorflow as tf

y_train_2d = tf.one_hot(y_train_2d, depth=2)
y_test_2d = tf.one_hot(y_test_2d, depth=2)
y_val_2d = tf.one_hot(y_val_2d, depth=2)

In [ ]:
y_train_2d.shape

TensorShape([8244, 2])

In [ ]:
from tensorflow.keras.layers import Conv1D, Conv2D, MaxPooling1D, MaxPooling2D, Concatenate
from datetime import datetime

from tensorflow.keras.layers import Input, DepthwiseConv2D
from tensorflow.keras.layers import Conv2D, BatchNormalization
from tensorflow.keras.layers import ReLU, Activation, AvgPool2D, Flatten, Dense
from tensorflow.keras.layers import Dropout, LSTM, GRU, Bidirectional

from tensorflow.keras import Model



# MobileNet block

def mobilnet_block (x, filters, strides):
    
    x = DepthwiseConv2D(kernel_size = 3, strides = strides, padding = 'same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    x = Conv2D(filters = filters, kernel_size = 1, strides = 1)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    return x

def hybrid_model(input_1, input_2):

  input_1 = Input(input_1[1:])

  x = Conv2D(filters = 32, kernel_size = 1, strides = 2, padding = 'same')(input_1)
  x = BatchNormalization()(x)
  x = ReLU()(x)

  # main part of the model

  x = mobilnet_block(x, filters = 64, strides = 1)
  x = mobilnet_block(x, filters = 128, strides = 2)
  x = mobilnet_block(x, filters = 128, strides = 1)
  x = mobilnet_block(x, filters = 256, strides = 2)
  x = mobilnet_block(x, filters = 256, strides = 1)
  x = mobilnet_block(x, filters = 512, strides = 2)

  for _ in range (5):
      x = mobilnet_block(x, filters = 512, strides = 1)
  x = mobilnet_block(x, filters = 1024, strides = 2)
  x = mobilnet_block(x, filters = 1024, strides = 1)
  x = AvgPool2D (pool_size = 7, strides = 1, data_format='channels_first')(x)

  x = Flatten()(x)
  x = Dense(units=500, activation='relu')(x)
  x = Dense(units=256, activation='relu')(x)
  x = Dense(units=128, activation='relu')(x)
  x = Dense(units=64, activation='relu')(x)
  # add meta
  # input_2 = Input((7380, 128000))
  input_2 = Input(input_2[1:])

  y = Conv1D(filters=64, kernel_size=(3), strides=1, padding='same')(input_2)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4)(y)
  y = Dropout(0.5)(y)

  # LFLB2
  y = Conv1D(filters=64, kernel_size=3, strides=1, padding='same')(y)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4, padding='same')(y)
  y = Dropout(0.25)(y)

  # LFLB3
  y = Conv1D(filters=128, kernel_size=3, strides=1, padding='same')(y)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4, padding='same')(y)
  y = Dropout(0.25)(y)

  # LFLB4
  y = Conv1D(filters=128, kernel_size=3, strides=1, padding='same')(y)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4, padding='same')(y)
  y = Dropout(0.25)(y)

  y = LSTM(units = 128, return_sequences = True, name='lstm_layer')(y)
  y = Dropout(rate = 0.2)(y) 
  y = BatchNormalization()(y)
  y = Dropout(rate = 0.2)(y)
  y = Flatten()(y)
  y = Dense(64)(y)


  merged = Concatenate()([x, y])
  output = Dense(2, activation='softmax')(merged)

  merged_model = Model(inputs=[input_1, input_2], outputs=[output])

  return merged_model

input_2d = X_train_2d.shape
input_1d = X_train_1d.shape
model = hybrid_model(input_2d, input_1d)

In [ ]:
LEARNING_RATE = 0.001
SAVED_MODEL_PATH = os.path.join(root, 'saved_model')
EPOCHS = 40
PATIENCE = 5


opt = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)
auc = tf.keras.metrics.AUC()
loss = tf.keras.losses.BinaryCrossentropy()

In [ ]:
model.compile(optimizer=opt,
              loss=loss,
              metrics=[auc])

In [ ]:
fp = f'./coswara_mfcc_{n_mfcc}_hybrid_bestweight.h5'

check = tf.keras.callbacks.ModelCheckpoint(fp,
                                           monitor='val_auc',
                                           save_best_only=True,
                                           mode='max')

earlystop_callback = tf.keras.callbacks.EarlyStopping(monitor="val_auc",
                                                      min_delta=0.001,
                                                      mode='max',
                                                      patience=PATIENCE)

log_dir = "logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [ ]:
for i in [8, 16, 32, 64, 128]:
  BATCH_SIZE = i

  history = model.fit([X_train_2d, X_train_1d],\
                      [y_train_2d, y_train_1d],\
                      epochs=EPOCHS,\
                      batch_size=BATCH_SIZE,\
                      validation_data=([X_val_2d, X_val_1d], [y_val_2d, y_val_1d]),\
                      callbacks=[check, earlystop_callback, tensorboard_callback],\
                      shuffle=True)
  
  model.evaluate([X_test_2d, X_test_1d], [y_test_2d, y_test_1d])

  model.save(os.path.join(SAVED_MODEL_PATH, f'std_full_audio_coughvid_mfcc_{n_mfcc}_hybrid_model_{EPOCHS}_{BATCH_SIZE}_{LEARNING_RATE}'))

  del model

  model = hybrid_model(input_2d, input_1d)

  model.compile(optimizer=opt,
               loss=loss,
               metrics=[auc])

Epoch 1/40
1031/1031 [==============================] - 89s 66ms/step - loss: 0.4947 - auc: 0.8372 - val_loss: 0.5196 - val_auc: 0.8304
Epoch 2/40
1031/1031 [==============================] - 62s 60ms/step - loss: 0.4670 - auc: 0.8380 - val_loss: 0.4633 - val_auc: 0.8286
Epoch 3/40
1031/1031 [==============================] - 63s 61ms/step - loss: 0.4598 - auc: 0.8421 - val_loss: 0.4786 - val_auc: 0.8277
Epoch 4/40
1031/1031 [==============================] - 64s 62ms/step - loss: 0.4600 - auc: 0.8396 - val_loss: 0.4600 - val_auc: 0.8318
Epoch 5/40
1031/1031 [==============================] - 64s 62ms/step - loss: 0.4568 - auc: 0.8427 - val_loss: 0.4545 - val_auc: 0.8387
Epoch 6/40
1031/1031 [==============================] - 64s 62ms/step - loss: 0.4573 - auc: 0.8392 - val_loss: 0.4677 - val_auc: 0.8307
Epoch 7/40
1031/1031 [==============================] - 67s 65ms/step - loss: 0.4558 - auc: 0.8433 - val_loss: 0.4578 - val_auc: 0.8425
Epoch 8/40
1031/1031 [==========================

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/std_full_audio_coughvid_mfcc_26_hybrid_model_40_8_0.001/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/std_full_audio_coughvid_mfcc_26_hybrid_model_40_8_0.001/assets


Epoch 1/40
516/516 [==============================] - 62s 109ms/step - loss: 0.6273 - auc: 0.8350 - val_loss: 0.4802 - val_auc: 0.8418
Epoch 2/40
516/516 [==============================] - 55s 106ms/step - loss: 0.4576 - auc: 0.8433 - val_loss: 0.4863 - val_auc: 0.8285
Epoch 3/40
516/516 [==============================] - 54s 105ms/step - loss: 0.4559 - auc: 0.8463 - val_loss: 0.4686 - val_auc: 0.8367
Epoch 4/40
516/516 [==============================] - 56s 108ms/step - loss: 0.4527 - auc: 0.8498 - val_loss: 0.4634 - val_auc: 0.8432
Epoch 5/40
516/516 [==============================] - 55s 107ms/step - loss: 0.4506 - auc: 0.8541 - val_loss: 0.4570 - val_auc: 0.8451
Epoch 6/40
516/516 [==============================] - 54s 105ms/step - loss: 0.4500 - auc: 0.8551 - val_loss: 0.4685 - val_auc: 0.8445
Epoch 7/40
516/516 [==============================] - 54s 105ms/step - loss: 0.4492 - auc: 0.8573 - val_loss: 0.4546 - val_auc: 0.8382
Epoch 8/40
516/516 [==============================] - 5

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/std_full_audio_coughvid_mfcc_26_hybrid_model_40_16_0.001/assets


...

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/std_full_audio_coughvid_mfcc_26_hybrid_model_40_64_0.001/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/std_full_audio_coughvid_mfcc_26_hybrid_model_40_64_0.001/assets


Epoch 1/40
65/65 [==============================] - 53s 708ms/step - loss: 0.7707 - auc: 0.8255 - val_loss: 0.5564 - val_auc: 0.8268
Epoch 2/40
65/65 [==============================] - 45s 686ms/step - loss: 0.4605 - auc: 0.8446 - val_loss: 0.6480 - val_auc: 0.8369
Epoch 3/40
65/65 [==============================] - 45s 701ms/step - loss: 0.4546 - auc: 0.8489 - val_loss: 0.6034 - val_auc: 0.8336
Epoch 4/40
65/65 [==============================] - 46s 703ms/step - loss: 0.4525 - auc: 0.8502 - val_loss: 0.5945 - val_auc: 0.8357
Epoch 5/40
65/65 [==============================] - 46s 705ms/step - loss: 0.4520 - auc: 0.8505 - val_loss: 0.5801 - val_auc: 0.7985
Epoch 6/40
65/65 [==============================] - 45s 700ms/step - loss: 0.4452 - auc: 0.8594 - val_loss: 0.6058 - val_auc: 0.8027
Epoch 7/40
33/33 [==============================] - 1s 28ms/step - loss: 0.5771 - auc: 0.8215


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/std_full_audio_coughvid_mfcc_26_hybrid_model_40_128_0.001/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/std_full_audio_coughvid_mfcc_26_hybrid_model_40_128_0.001/assets


## SMOTE

In [ ]:
X_train_2d.shape

(8244, 234, 246)

In [ ]:
from imblearn.over_sampling import SMOTE


sm = SMOTE(random_state=42)

num_instances, first_dim, second_dim = X_train_2d.shape
X_train_2d = np.reshape(X_train_2d, newshape=(num_instances, -1))
X_train_2d, y_train_2d = sm.fit_resample(X_train_2d, y_train_2d)
X_train_2d = np.reshape(X_train_2d, newshape=(X_train_2d.shape[0], first_dim, second_dim))


num_instances, first_dim = X_train_1d.shape
X_train_1d = np.reshape(X_train_1d, newshape=(num_instances, -1))
X_train_1d, y_train_1d = sm.fit_resample(X_train_1d, y_train_1d)
X_train_1d = np.reshape(X_train_1d, newshape=(X_train_1d.shape[0], first_dim))

# num_instances, first_dim, second_dim = X_val.shape
# X_val = np.reshape(X_val, newshape=(-1, second_dim))
# X_val = sc.transform(X_val)
# X_val = np.reshape(X_val, newshape=(num_instances, first_dim, second_dim))


# num_instances, first_dim, second_dim = X_test.shape
# X_test = np.reshape(X_test, newshape=(-1, second_dim))
# X_test = sc.transform(X_test)
# X_test = np.reshape(X_test, newshape=(num_instances, first_dim, second_dim))

# reshape training data

X_train_2d, X_val_2d, X_test_2d = X_train_2d[..., np.newaxis], X_val_2d[..., np.newaxis], X_test_2d[..., np.newaxis]
X_train_2d.shape

(13700, 234, 246, 1)

In [ ]:
X_train_1d, X_val_1d, X_test_1d = X_train_1d[..., np.newaxis], X_val_1d[..., np.newaxis], X_test_1d[..., np.newaxis]
X_train_1d.shape

(13700, 37, 1)

In [ ]:
# reshape target
import tensorflow as tf

y_train_2d = tf.one_hot(y_train_2d, depth=2)
y_test_2d = tf.one_hot(y_test_2d, depth=2)
y_val_2d = tf.one_hot(y_val_2d, depth=2)

In [ ]:
y_train_2d.shape

TensorShape([13700, 2])

In [ ]:
SAVED_MODEL_PATH = os.path.join(root, 'saved_model')
EPOCHS = 40
PATIENCE = 5
LEARNING_RATE = 0.001

In [ ]:
from tensorflow.keras.layers import Conv1D, Conv2D, MaxPooling1D, MaxPooling2D, Concatenate
from datetime import datetime

from tensorflow.keras.layers import Input, DepthwiseConv2D
from tensorflow.keras.layers import Conv2D, BatchNormalization
from tensorflow.keras.layers import ReLU, Activation, AvgPool2D, Flatten, Dense
from tensorflow.keras.layers import Dropout, LSTM, GRU, Bidirectional

from tensorflow.keras import Model



# MobileNet block

def mobilnet_block (x, filters, strides):
    
    x = DepthwiseConv2D(kernel_size = 3, strides = strides, padding = 'same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    x = Conv2D(filters = filters, kernel_size = 1, strides = 1)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    return x

def hybrid_model(input_1, input_2):

  input_1 = Input(input_1[1:])

  x = Conv2D(filters = 32, kernel_size = 1, strides = 2, padding = 'same')(input_1)
  x = BatchNormalization()(x)
  x = ReLU()(x)

  # main part of the model

  x = mobilnet_block(x, filters = 64, strides = 1)
  x = mobilnet_block(x, filters = 128, strides = 2)
  x = mobilnet_block(x, filters = 128, strides = 1)
  x = mobilnet_block(x, filters = 256, strides = 2)
  x = mobilnet_block(x, filters = 256, strides = 1)
  x = mobilnet_block(x, filters = 512, strides = 2)

  for _ in range (5):
      x = mobilnet_block(x, filters = 512, strides = 1)
  x = mobilnet_block(x, filters = 1024, strides = 2)
  x = mobilnet_block(x, filters = 1024, strides = 1)
  x = AvgPool2D (pool_size = 7, strides = 1, data_format='channels_first')(x)

  x = Flatten()(x)
  x = Dense(units=500, activation='relu')(x)
  x = Dense(units=256, activation='relu')(x)
  x = Dense(units=128, activation='relu')(x)
  x = Dense(units=64, activation='relu')(x)
  # add meta
  # input_2 = Input((7380, 128000))
  input_2 = Input(input_2[1:])

  y = Conv1D(filters=64, kernel_size=(3), strides=1, padding='same')(input_2)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4)(y)
  y = Dropout(0.5)(y)

  # LFLB2
  y = Conv1D(filters=64, kernel_size=3, strides=1, padding='same')(y)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4, padding='same')(y)
  y = Dropout(0.25)(y)

  # LFLB3
  y = Conv1D(filters=128, kernel_size=3, strides=1, padding='same')(y)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4, padding='same')(y)
  y = Dropout(0.25)(y)

  # LFLB4
  y = Conv1D(filters=128, kernel_size=3, strides=1, padding='same')(y)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4, padding='same')(y)
  y = Dropout(0.25)(y)

  y = LSTM(units = 128, return_sequences = True, name='lstm_layer')(y)
  y = Dropout(rate = 0.2)(y) 
  y = BatchNormalization()(y)
  y = Dropout(rate = 0.2)(y)
  y = Flatten()(y)
  y = Dense(64)(y)


  merged = Concatenate()([x, y])
  output = Dense(2, activation='softmax')(merged)

  merged_model = Model(inputs=[input_1, input_2], outputs=[output])

  return merged_model

input_2d = X_train_2d.shape
input_1d = X_train_1d.shape
model = hybrid_model(input_2d, input_1d)

In [ ]:
opt = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)
auc = tf.keras.metrics.AUC()
loss = tf.keras.losses.BinaryCrossentropy()
# model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.compile(optimizer=opt,
              loss=loss,
              metrics=[auc])

In [ ]:
fp = f'./coswara_mfcc_{n_mfcc}_bestweight.h5'

check = tf.keras.callbacks.ModelCheckpoint(fp,
                                           monitor='val_auc',
                                           save_best_only=True,
                                           mode='max')

earlystop_callback = tf.keras.callbacks.EarlyStopping(monitor="val_auc",
                                                      min_delta=0.001,
                                                      mode='max',
                                                      patience=PATIENCE)

log_dir = "logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [ ]:
for i in [8, 16, 32, 64, 128]:
  BATCH_SIZE = i

  history = model.fit([X_train_2d, X_train_1d],\
                      [y_train_2d, y_train_1d],\
                      epochs=EPOCHS,\
                      batch_size=BATCH_SIZE,\
                      validation_data=([X_val_2d, X_val_1d], [y_val_2d, y_val_1d]),\
                      callbacks=[check, earlystop_callback, tensorboard_callback],\
                      shuffle=True)
  
  model.evaluate([X_test_2d, X_test_1d], [y_test_2d, y_test_1d])

  model.save(os.path.join(SAVED_MODEL_PATH, f'smote_full_audio_coughvid_mfcc_{n_mfcc}_hybrid_model_{EPOCHS}_{BATCH_SIZE}_{LEARNING_RATE}'))

  del model

  model = hybrid_model(input_2d, input_1d)

  model.compile(optimizer=opt,
               loss=loss,
               metrics=[auc])

Epoch 1/40
1713/1713 [==============================] - 115s 61ms/step - loss: 0.6284 - auc: 0.7265 - val_loss: 0.5423 - val_auc: 0.8338
Epoch 2/40
1713/1713 [==============================] - 101s 59ms/step - loss: 0.5613 - auc: 0.7842 - val_loss: 0.5337 - val_auc: 0.8080
Epoch 3/40
1713/1713 [==============================] - 102s 60ms/step - loss: 0.5351 - auc: 0.8069 - val_loss: 0.8232 - val_auc: 0.5882
Epoch 4/40
1713/1713 [==============================] - 99s 58ms/step - loss: 0.5163 - auc: 0.8226 - val_loss: 0.5179 - val_auc: 0.8250
Epoch 5/40
1713/1713 [==============================] - 102s 60ms/step - loss: 0.5028 - auc: 0.8326 - val_loss: 1.3271 - val_auc: 0.3842
Epoch 6/40
1713/1713 [==============================] - 102s 60ms/step - loss: 0.4813 - auc: 0.8480 - val_loss: 0.4818 - val_auc: 0.8411
Epoch 7/40
1713/1713 [==============================] - 102s 59ms/step - loss: 0.4519 - auc: 0.8680 - val_loss: 0.5661 - val_auc: 0.7853
Epoch 8/40
1713/1713 [====================

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/smote_full_audio_coughvid_mfcc_26_hybrid_model_40_8_0.001/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/smote_full_audio_coughvid_mfcc_26_hybrid_model_40_8_0.001/assets


Epoch 1/40
857/857 [==============================] - 96s 104ms/step - loss: 0.6897 - auc: 0.6965 - val_loss: 1.5562 - val_auc: 0.3431
Epoch 2/40
857/857 [==============================] - 88s 103ms/step - loss: 0.5317 - auc: 0.8090 - val_loss: 0.7030 - val_auc: 0.6854
Epoch 3/40
857/857 [==============================] - 89s 103ms/step - loss: 0.4990 - auc: 0.8351 - val_loss: 0.9439 - val_auc: 0.5339
Epoch 4/40
857/857 [==============================] - 88s 103ms/step - loss: 0.4705 - auc: 0.8555 - val_loss: 0.4753 - val_auc: 0.8360
Epoch 5/40
857/857 [==============================] - 89s 103ms/step - loss: 0.4549 - auc: 0.8660 - val_loss: 0.4646 - val_auc: 0.8351
Epoch 6/40
857/857 [==============================] - 89s 104ms/step - loss: 0.4232 - auc: 0.8846 - val_loss: 6.6375 - val_auc: 0.1794
Epoch 7/40
857/857 [==============================] - 90s 105ms/step - loss: 0.4049 - auc: 0.8951 - val_loss: 0.5859 - val_auc: 0.8421
Epoch 8/40
857/857 [==============================] - 8

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/smote_full_audio_coughvid_mfcc_26_hybrid_model_40_16_0.001/assets


...

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/smote_full_audio_coughvid_mfcc_26_hybrid_model_40_64_0.001/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/smote_full_audio_coughvid_mfcc_26_hybrid_model_40_64_0.001/assets


Epoch 1/40
108/108 [==============================] - 84s 713ms/step - loss: 0.9625 - auc: 0.6982 - val_loss: 0.6351 - val_auc: 0.8396
Epoch 2/40
108/108 [==============================] - 75s 698ms/step - loss: 0.5155 - auc: 0.8199 - val_loss: 0.6361 - val_auc: 0.8050
Epoch 3/40
108/108 [==============================] - 75s 699ms/step - loss: 0.4642 - auc: 0.8569 - val_loss: 0.6281 - val_auc: 0.7700
Epoch 4/40
108/108 [==============================] - 75s 698ms/step - loss: 0.5215 - auc: 0.8149 - val_loss: 0.7213 - val_auc: 0.5165
Epoch 5/40
108/108 [==============================] - 75s 699ms/step - loss: 0.4647 - auc: 0.8583 - val_loss: 1.1749 - val_auc: 0.1740
Epoch 6/40
33/33 [==============================] - 1s 29ms/step - loss: 1.8689 - auc: 0.1867


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/smote_full_audio_coughvid_mfcc_26_hybrid_model_40_128_0.001/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/smote_full_audio_coughvid_mfcc_26_hybrid_model_40_128_0.001/assets


## Evaluation

In [ ]:
SAVED_MODEL_PATH = os.path.join(root, 'saved_model')

### Standard scaler

In [ ]:
X_train_1d, X_val_1d, X_test_1d = X_train_1d[..., np.newaxis], X_val_1d[..., np.newaxis], X_test_1d[..., np.newaxis]

In [ ]:
from sklearn.preprocessing import StandardScaler


sc = StandardScaler()

num_instances, first_dim, second_dim = X_train_2d.shape
X_train_std = np.reshape(X_train_2d, newshape=(-1, second_dim))
X_train_std = sc.fit_transform(X_train_std)
X_train_std = np.reshape(X_train_std, newshape=(num_instances, first_dim, second_dim))


num_instances, first_dim, second_dim = X_val_2d.shape
X_val_std = np.reshape(X_val_2d, newshape=(-1, second_dim))
X_val_std = sc.transform(X_val_std)
X_val_std = np.reshape(X_val_std, newshape=(num_instances, first_dim, second_dim))


num_instances, first_dim, second_dim = X_test_2d.shape
X_test_std = np.reshape(X_test_2d, newshape=(-1, second_dim))
X_test_std = sc.transform(X_test_std)
X_test_std = np.reshape(X_test_std, newshape=(num_instances, first_dim, second_dim))

# reshape training data

X_train_std, X_val_std, X_test_std = X_train_std[..., np.newaxis], X_val_std[..., np.newaxis], X_test_std[..., np.newaxis]

# reshape target
import tensorflow as tf

y_train_std = tf.one_hot(y_train_2d, depth=2)
y_test_std = tf.one_hot(y_test_2d, depth=2)
y_val_std = tf.one_hot(y_val_2d, depth=2)

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'std_full_audio_coughvid_mfcc_{n_mfcc}_hybrid_model_40_128_0.001'))
loaded_model.evaluate([X_test_std, X_test_1d], [y_test_std, y_test_1d], verbose=1)

33/33 [==============================] - 8s 34ms/step - loss: 0.5771 - auc: 0.8215


[0.5770652890205383, 0.8214982151985168]

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'std_full_audio_coughvid_mfcc_{n_mfcc}_hybrid_model_40_64_0.001'))
loaded_model.evaluate([X_test_std, X_test_1d], [y_test_std, y_test_1d], verbose=1)

33/33 [==============================] - 2s 29ms/step - loss: 0.4768 - auc: 0.8470


[0.4768354296684265, 0.8469936847686768]

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'std_full_audio_coughvid_mfcc_{n_mfcc}_hybrid_model_40_32_0.001'))
loaded_model.evaluate([X_test_std, X_test_1d], [y_test_std, y_test_1d], verbose=1)

33/33 [==============================] - 2s 29ms/step - loss: 0.4590 - auc: 0.8448


[0.4589981138706207, 0.8447695970535278]

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'std_full_audio_coughvid_mfcc_{n_mfcc}_hybrid_model_40_16_0.001'))
loaded_model.evaluate([X_test_std, X_test_1d], [y_test_std, y_test_1d], verbose=1)

33/33 [==============================] - 2s 31ms/step - loss: 0.4612 - auc: 0.8469


[0.46121037006378174, 0.8468706011772156]

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'std_full_audio_coughvid_mfcc_{n_mfcc}_hybrid_model_40_8_0.001'))
loaded_model.evaluate([X_test_std, X_test_1d], [y_test_std, y_test_1d], verbose=1)

33/33 [==============================] - 2s 31ms/step - loss: 0.4504 - auc: 0.8482


[0.45043691992759705, 0.8482477068901062]

### Raw features

In [ ]:
# reshape training data

X_train_2d, X_val_2d, X_test_2d = X_train_2d[..., np.newaxis], X_val_2d[..., np.newaxis], X_test_2d[..., np.newaxis]
X_train_2d.shape

# reshape target
import tensorflow as tf

y_train = tf.one_hot(y_train_2d, depth=2)
y_test = tf.one_hot(y_test_2d, depth=2)
y_val = tf.one_hot(y_val_2d, depth=2)

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'full_audio_coughvid_mfcc_{n_mfcc}_hybrid_model_40_128_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

33/33 [==============================] - 2s 31ms/step - loss: 0.5463 - auc: 0.8037


[0.5462851524353027, 0.8036874532699585]

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'full_audio_coughvid_mfcc_{n_mfcc}_hybrid_model_40_64_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

33/33 [==============================] - 2s 35ms/step - loss: 0.4610 - auc: 0.8477


[0.4610477685928345, 0.8476897478103638]

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'full_audio_coughvid_mfcc_{n_mfcc}_hybrid_model_40_32_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

33/33 [==============================] - 2s 31ms/step - loss: 0.4609 - auc: 0.8446


[0.4609357714653015, 0.8446016907691956]

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'full_audio_coughvid_mfcc_{n_mfcc}_hybrid_model_40_16_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

33/33 [==============================] - 2s 32ms/step - loss: 0.4688 - auc: 0.8425


[0.46875619888305664, 0.8424555063247681]

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'full_audio_coughvid_mfcc_{n_mfcc}_hybrid_model_40_8_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

33/33 [==============================] - 2s 31ms/step - loss: 0.4559 - auc: 0.8396


[0.45592644810676575, 0.8396323919296265]

### SMOTE

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'smote_full_audio_coughvid_mfcc_{n_mfcc}_hybrid_model_40_128_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

33/33 [==============================] - 2s 31ms/step - loss: 1.8689 - auc: 0.1867


[1.8688684701919556, 0.18665283918380737]

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'smote_full_audio_coughvid_mfcc_{n_mfcc}_hybrid_model_40_64_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

33/33 [==============================] - 3s 31ms/step - loss: 1.1767 - auc: 0.8337


[1.1766941547393799, 0.8337196111679077]

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'smote_full_audio_coughvid_mfcc_{n_mfcc}_hybrid_model_40_32_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

33/33 [==============================] - 3s 38ms/step - loss: 1.1494 - auc: 0.5652


[1.149422287940979, 0.5651782155036926]

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'smote_full_audio_coughvid_mfcc_{n_mfcc}_hybrid_model_40_16_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

33/33 [==============================] - 2s 31ms/step - loss: 1.1588 - auc: 0.6841


[1.158835530281067, 0.684078574180603]

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'smote_full_audio_coughvid_mfcc_{n_mfcc}_hybrid_model_40_8_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

33/33 [==============================] - 2s 31ms/step - loss: 0.6660 - auc: 0.7681


[0.6660033464431763, 0.7681435346603394]

# Chunks
* potentials: use `num_chunk` to explore as long audio influence the model'accuracy (the more `num_chunk` the longer the audio)

## Feature

### Set up

In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
from IPython.display import clear_output, Audio
import IPython.display as ipd
import os
import numpy as np
import shutil
import pandas as pd
import logging
import warnings

warnings.filterwarnings("ignore")


logger = logging.getLogger('COUGHVID - HYBRID MODEL CHUNKS')
logger.setLevel(logging.DEBUG)

# create console handler and set level to debug
ch = logging.StreamHandler()
ch.setLevel(logging.DEBUG)

# create formatter
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')

# add formatter to ch
ch.setFormatter(formatter)

# add ch to logger
logger.addHandler(ch)
pd.set_option('display.max_colwidth', None)
clear_output()

In [3]:
n_mfcc = 26

root = '/content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet'
data_root = '/content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/data preprocessing'

meta_coughvid = os.path.join(data_root, 'cough_chunking/coughvid/coughvid_chunk_metadata.csv')

coughvid_feature = [
                    os.path.join(root, f'chunks/mfcc_{n_mfcc}_coughvid_features_20k.npy'),
                    os.path.join(root, f'chunks/mfcc_{n_mfcc}_coughvid_features_20k_40k.npy'),
                    os.path.join(root, f'chunks/mfcc_{n_mfcc}_coughvid_features_40k_60k.npy'),
                    os.path.join(root, f'chunks/mfcc_{n_mfcc}_coughvid_features_60k_75k.npy'),
                    os.path.join(root, f'chunks/mfcc_{n_mfcc}_coughvid_features_9981.npy'),
]


### Feature preparation

#### Metadata

In [4]:
# process metadata
meta_df = pd.read_csv(meta_coughvid)
meta_df.head()

,uuid,datetime,cough_detected,latitude,longitude,age,gender,respiratory_condition,fever_muscle_pain,quality_1,...,choking_3,congestion_3,nothing_3,cough_type_3,diagnosis_3,severity_3,label,sr_cough,chunk_path,num_chunk
0,00039425-7f3a-42aa-ac13-834aaa2b6b92-0,2020-04-13t21:30:59.801831+00:00,0.9609,31.3,34.8,15.0,male,False,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,healthy,coughvid_cough_data/00039425-7f3a-42aa-ac13-834aaa2b6b92.webm,coughvid_cough_chunk/00039425-7f3a-42aa-ac13-834aaa2b6b92-0.wav,3
1,00039425-7f3a-42aa-ac13-834aaa2b6b92-1,2020-04-13t21:30:59.801831+00:00,0.9609,31.3,34.8,15.0,male,False,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,healthy,coughvid_cough_data/00039425-7f3a-42aa-ac13-834aaa2b6b92.webm,coughvid_cough_chunk/00039425-7f3a-42aa-ac13-834aaa2b6b92-1.wav,3
2,00039425-7f3a-42aa-ac13-834aaa2b6b92-2,2020-04-13t21:30:59.801831+00:00,0.9609,31.3,34.8,15.0,male,False,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,healthy,coughvid_cough_data/00039425-7f3a-42aa-ac13-834aaa2b6b92.webm,coughvid_cough_chunk/00039425-7f3a-42aa-ac13-834aaa2b6b92-2.wav,3
3,0009eb28-d8be-4dc1-92bb-907e53bc5c7a-0,2020-04-12t04:02:18.159383+00:00,0.9301,40.0,-75.1,34.0,male,True,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,healthy,coughvid_cough_data/0009eb28-d8be-4dc1-92bb-907e53bc5c7a.webm,coughvid_cough_chunk/0009eb28-d8be-4dc1-92bb-907e53bc5c7a-0.wav,3
4,0009eb28-d8be-4dc1-92bb-907e53bc5c7a-1,2020-04-12t04:02:18.159383+00:00,0.9301,40.0,-75.1,34.0,male,True,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,healthy,coughvid_cough_data/0009eb28-d8be-4dc1-92bb-907e53bc5c7a.webm,coughvid_cough_chunk/0009eb28-d8be-4dc1-92bb-907e53bc5c7a-1.wav,3


In [5]:
logger.info(meta_df.shape)

2022-05-02 23:22:19,139 - COUGHVID - HYBRID MODEL CHUNKS - INFO - (84981, 43)


In [6]:
DROP_COLS = [
             'uuid',
             'num_chunk',
             'datetime',
             'chunk_path',
             'sr_cough',
             'cough_detected',
             ]

meta_df.isna().sum()

uuid                         0
datetime                     0
cough_detected               0
latitude                 33197
longitude                33197
age                      35707
gender                   31986
respiratory_condition    31986
fever_muscle_pain        31986
quality_1                80743
cough_type_1             80743
dyspnea_1                80743
wheezing_1               80743
stridor_1                80743
choking_1                80743
congestion_1             80743
nothing_1                80743
diagnosis_1              80743
severity_1               80743
quality_2                80643
cough_type_2             80651
dyspnea_2                80643
wheezing_2               80643
stridor_2                80643
choking_2                80643
congestion_2             80643
nothing_2                80643
diagnosis_2              80643
severity_2               80643
quality_3                80594
dyspnea_3                80594
wheezing_3               80594
stridor_

In [7]:
meta_df.dtypes

uuid                      object
datetime                  object
cough_detected           float64
latitude                 float64
longitude                float64
age                      float64
gender                    object
respiratory_condition     object
fever_muscle_pain         object
quality_1                 object
cough_type_1              object
dyspnea_1                 object
wheezing_1                object
stridor_1                 object
choking_1                 object
congestion_1              object
nothing_1                 object
diagnosis_1               object
severity_1                object
quality_2                 object
cough_type_2              object
dyspnea_2                 object
wheezing_2                object
stridor_2                 object
choking_2                 object
congestion_2              object
nothing_2                 object
diagnosis_2               object
severity_2                object
quality_3                 object
dyspnea_3 

In [8]:
# fill na with unknown value
# meta_df[['latitude', 'longitude', 'age']] = meta_df[['latitude', 'longitude', 'age']].fillna(99999)
meta_df.fillna('unknown', inplace=True)
meta_df.isna().sum()

uuid                     0
datetime                 0
cough_detected           0
latitude                 0
longitude                0
age                      0
gender                   0
respiratory_condition    0
fever_muscle_pain        0
quality_1                0
cough_type_1             0
dyspnea_1                0
wheezing_1               0
stridor_1                0
choking_1                0
congestion_1             0
nothing_1                0
diagnosis_1              0
severity_1               0
quality_2                0
cough_type_2             0
dyspnea_2                0
wheezing_2               0
stridor_2                0
choking_2                0
congestion_2             0
nothing_2                0
diagnosis_2              0
severity_2               0
quality_3                0
dyspnea_3                0
wheezing_3               0
stridor_3                0
choking_3                0
congestion_3             0
nothing_3                0
cough_type_3             0
d

In [9]:
# drop cols
meta_df.drop(DROP_COLS, axis=1, inplace=True)

In [10]:
# take labels
meta_df.label = meta_df.label.map({
    'healthy': 0,
    'symptomatic': 1,
    'covid-19': 1
})
logger.info(meta_df.label.value_counts().to_dict())

2022-05-02 23:22:19,685 - COUGHVID - HYBRID MODEL CHUNKS - INFO - {0.0: 39866, 1.0: 13129}


In [11]:
meta_df.label.value_counts().sum()

52995

In [12]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

for col in meta_df.drop(['label'], axis=1).columns:
  meta_df[col] = meta_df[col].astype('string')
  meta_df[col] = le.fit_transform(meta_df[col].values)

In [13]:
meta_feature = np.array(meta_df.drop(['label'], axis=1))

#### Features

In [14]:
mfcc_feature_1 = np.load(coughvid_feature[0])
mfcc_feature_2 = np.load(coughvid_feature[1])
mfcc_feature_3 = np.load(coughvid_feature[2])
mfcc_feature_4 = np.load(coughvid_feature[3])
mfcc_feature_5 = np.load(coughvid_feature[4])

##### 2d

In [15]:
null_label = meta_df[meta_df['label'].isna() == False].index.tolist() # index of records not n/a labels
meta_df.dropna(inplace=True)
meta_df.reset_index(drop=True, inplace=True)

In [16]:
label_0 = meta_df[meta_df['label']==0].copy().reset_index(drop=True)
label_1 = meta_df[meta_df['label']==1].copy().reset_index(drop=True)

In [17]:
# get index of each label 0, 1 to match with extracted features
index_1 = meta_df[meta_df['label']==1].index.tolist()
index_0 = meta_df[meta_df['label']==0].index.tolist()
logger.info(f'{len(index_1)}, {len(index_0)}')

2022-05-02 23:23:36,074 - COUGHVID - HYBRID MODEL CHUNKS - INFO - 13129, 39866


In [18]:
# remove features of null labels
mfcc_feature_1 = mfcc_feature_1[[i for i in null_label if i < 20000]]
mfcc_feature_2 = mfcc_feature_2[[i - 20000 for i in null_label if i >= 20000 and i < 40000]]
mfcc_feature_3 = mfcc_feature_3[[i - 40000 for i in null_label if i >= 40000 and i < 60000]]
mfcc_feature_4 = mfcc_feature_4[[i - 60000 for i in null_label if i >= 60000 and i < 75000]]
mfcc_feature_5 = mfcc_feature_5[[i - 75000 for i in null_label if i >= 75000]]
mfcc_features = np.concatenate((mfcc_feature_1, mfcc_feature_2, mfcc_feature_3, mfcc_feature_4, mfcc_feature_5), axis=0)

In [19]:
# release RAM
del mfcc_feature_1
del mfcc_feature_2
del mfcc_feature_3
del mfcc_feature_4
del mfcc_feature_5

In [20]:
# get index of each label 0, 1 to match with extracted features
index_1 = meta_df[meta_df['label']==1].index.tolist()
index_0 = meta_df[meta_df['label']==0].index.tolist()
logger.info(f'{len(index_0)}, {len(index_1)}')

2022-05-02 23:23:55,524 - COUGHVID - HYBRID MODEL CHUNKS - INFO - 39866, 13129


In [21]:
mfcc_features.shape

(52995, 234, 236)

In [22]:
# get features for each class
feature_0 = mfcc_features[index_0]
feature_1 = mfcc_features[index_1]
del mfcc_features

In [23]:
# split data and reshape feature
def data_split(feature_0, feature_1, label0, label1):
  train = np.concatenate((feature_0[:31893], feature_1[:10517]), axis=0)
  val = np.concatenate((feature_0[31893:35879], feature_1[10517:11824]), axis=0)
  test = np.concatenate((feature_0[35879:], feature_1[11824:]), axis=0)

  y_train = pd.concat([label_0[:31893], label_1[:10517]], ignore_index=True)
  y_val = pd.concat([label_0[31893:35879], label_1[10517:11824]], ignore_index=True)
  y_test = pd.concat([label_0[35879:], label_1[11824:]], ignore_index=True)

  return train, val, test,\
         np.array(y_train.label), np.array(y_val.label), np.array(y_test.label)

In [24]:
X_train_2d, X_val_2d, X_test_2d, y_train_2d, y_val_2d, y_test_2d = data_split(feature_0, feature_1, label_0, label_1)
print(X_train_2d.shape)

(42410, 234, 236)


In [25]:
y_train_2d

array([0., 0., 0., ..., 1., 1., 1.])

In [26]:
del feature_0
del feature_1

##### 1d

In [27]:
feature_0 = meta_feature[index_0]
feature_1 = meta_feature[index_1]

In [28]:
X_train_1d, X_val_1d, X_test_1d, y_train_1d, y_val_1d, y_test_1d = data_split(feature_0, feature_1, label_0, label_1)

In [29]:
X_train_2d = X_train_2d[20000:]
y_train_2d = y_train_2d[20000:]
X_train_2d.shape, y_train_2d.shape

((22410, 234, 236), (22410,))

In [30]:
X_train_1d = X_train_1d[20000:]
y_train_1d = y_train_1d[20000:]
X_train_1d.shape, y_train_1d.shape

((22410, 36), (22410,))

In [31]:
from sklearn.utils import shuffle

X_train_2d, X_train_1d, y_train_2d, y_train_1d = shuffle(X_train_2d, X_train_1d, y_train_2d, y_train_1d, random_state=42)
X_val_2d, X_val_1d, y_val_2d, y_val_1d = shuffle(X_val_2d, X_val_1d, y_val_2d, y_val_1d, random_state=42)
X_test_2d, X_test_1d, y_test_2d, y_test_1d = shuffle(X_test_2d, X_test_1d, y_test_2d, y_test_1d, random_state=42)

## Model

In [33]:
X_train_2d, X_val_2d, X_test_2d = X_train_2d[..., np.newaxis], X_val_2d[..., np.newaxis], X_test_2d[..., np.newaxis]
X_train_2d.shape

(32410, 234, 236, 1)

In [34]:
X_train_1d, X_val_1d, X_test_1d = X_train_1d[..., np.newaxis], X_val_1d[..., np.newaxis], X_test_1d[..., np.newaxis]
X_train_1d.shape

(32410, 36, 1)

In [35]:
X_train_1d.shape

(32410, 36, 1)

In [36]:
import tensorflow as tf

y_train_2d = tf.one_hot(y_train_2d, depth=2)
y_test_2d = tf.one_hot(y_test_2d, depth=2)
y_val_2d = tf.one_hot(y_val_2d, depth=2)

In [37]:
SAVED_MODEL_PATH = os.path.join(root, 'saved_model')
EPOCHS = 40
PATIENCE = 5
LEARNING_RATE = 0.001

In [38]:
from tensorflow.keras.layers import Conv1D, Conv2D, MaxPooling1D, MaxPooling2D, Concatenate
from datetime import datetime

from tensorflow.keras.layers import Input, DepthwiseConv2D
from tensorflow.keras.layers import Conv2D, BatchNormalization
from tensorflow.keras.layers import ReLU, Activation, AvgPool2D, Flatten, Dense
from tensorflow.keras.layers import Dropout, LSTM, GRU, Bidirectional

from tensorflow.keras import Model



# MobileNet block

def mobilnet_block (x, filters, strides):
    
    x = DepthwiseConv2D(kernel_size = 3, strides = strides, padding = 'same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    x = Conv2D(filters = filters, kernel_size = 1, strides = 1)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    return x

def hybrid_model(input_1, input_2):

  input_1 = Input(input_1[1:])

  x = Conv2D(filters = 32, kernel_size = 1, strides = 2, padding = 'same')(input_1)
  x = BatchNormalization()(x)
  x = ReLU()(x)

  # main part of the model

  x = mobilnet_block(x, filters = 64, strides = 1)
  x = mobilnet_block(x, filters = 128, strides = 2)
  x = mobilnet_block(x, filters = 128, strides = 1)
  x = mobilnet_block(x, filters = 256, strides = 2)
  x = mobilnet_block(x, filters = 256, strides = 1)
  x = mobilnet_block(x, filters = 512, strides = 2)

  for _ in range (5):
      x = mobilnet_block(x, filters = 512, strides = 1)
  x = mobilnet_block(x, filters = 1024, strides = 2)
  x = mobilnet_block(x, filters = 1024, strides = 1)
  x = AvgPool2D (pool_size = 7, strides = 1, data_format='channels_first')(x)

  x = Flatten()(x)
  x = Dense(units=500, activation='relu')(x)
  x = Dense(units=256, activation='relu')(x)
  x = Dense(units=128, activation='relu')(x)
  x = Dense(units=64, activation='relu')(x)
  # add meta
  # input_2 = Input((7380, 128000))
  input_2 = Input(input_2[1:])

  y = Conv1D(filters=64, kernel_size=(3), strides=1, padding='same')(input_2)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4)(y)
  y = Dropout(0.5)(y)

  # LFLB2
  y = Conv1D(filters=64, kernel_size=3, strides=1, padding='same')(y)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4, padding='same')(y)
  y = Dropout(0.25)(y)

  # LFLB3
  y = Conv1D(filters=128, kernel_size=3, strides=1, padding='same')(y)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4, padding='same')(y)
  y = Dropout(0.25)(y)

  # LFLB4
  y = Conv1D(filters=128, kernel_size=3, strides=1, padding='same')(y)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4, padding='same')(y)
  y = Dropout(0.25)(y)

  y = LSTM(units = 128, return_sequences = True, name='lstm_layer')(y)
  y = Dropout(rate = 0.2)(y) 
  y = BatchNormalization()(y)
  y = Dropout(rate = 0.2)(y)
  y = Flatten()(y)
  y = Dense(64)(y)


  merged = Concatenate()([x, y])
  output = Dense(2, activation='softmax')(merged)

  merged_model = Model(inputs=[input_1, input_2], outputs=[output])

  return merged_model

input_2d = X_train_2d.shape
input_1d = X_train_1d.shape
model = hybrid_model(input_2d, input_1d)

In [39]:
opt = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)
auc = tf.keras.metrics.AUC()
loss = tf.keras.losses.BinaryCrossentropy()

model.compile(optimizer=opt,
              loss=loss,
              metrics=[auc])

In [40]:
fp = './bestweight.h5'

check = tf.keras.callbacks.ModelCheckpoint(fp,
                                           monitor='val_auc',
                                           save_best_only=True,
                                           mode='max')

earlystop_callback = tf.keras.callbacks.EarlyStopping(monitor="val_auc",
                                                      min_delta=0.001,
                                                      mode='max',
                                                      patience=5)

log_dir = "logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [41]:
for i in [8, 16, 32, 64, 128]:
  BATCH_SIZE = i

  history = model.fit([X_train_2d, X_train_1d],\
                      [y_train_2d, y_train_1d],\
                      epochs=EPOCHS,\
                      batch_size=BATCH_SIZE,\
                      validation_data=([X_val_2d, X_val_1d], [y_val_2d, y_val_1d]),\
                      callbacks=[check, earlystop_callback, tensorboard_callback],\
                      shuffle=True)
  
  model.evaluate([X_test_2d, X_test_1d], [y_test_2d, y_test_1d])

  model.save(os.path.join(SAVED_MODEL_PATH, f'coughvid_mfcc_{n_mfcc}_hybrid_model_{EPOCHS}_{BATCH_SIZE}_{LEARNING_RATE}'))

  del model

  model = hybrid_model(input_2d, input_1d)

  model.compile(optimizer=opt,
               loss=loss,
               metrics=[auc])

Epoch 1/40
4052/4052 [==============================] - 262s 60ms/step - loss: 0.6500 - auc: 0.6722 - val_loss: 0.5799 - val_auc: 0.7457
Epoch 2/40
4052/4052 [==============================] - 242s 60ms/step - loss: 0.6333 - auc: 0.6755 - val_loss: 0.5679 - val_auc: 0.7485
Epoch 3/40
4052/4052 [==============================] - 242s 60ms/step - loss: 0.6327 - auc: 0.6766 - val_loss: 0.5812 - val_auc: 0.7415
Epoch 4/40
4052/4052 [==============================] - 245s 60ms/step - loss: 0.6323 - auc: 0.6774 - val_loss: 0.5842 - val_auc: 0.7397
Epoch 5/40
4052/4052 [==============================] - 249s 62ms/step - loss: 0.6322 - auc: 0.6764 - val_loss: 0.5682 - val_auc: 0.7575
Epoch 6/40
4052/4052 [==============================] - 244s 60ms/step - loss: 0.6323 - auc: 0.6740 - val_loss: 0.5706 - val_auc: 0.7651
Epoch 7/40
4052/4052 [==============================] - 258s 64ms/step - loss: 0.6319 - auc: 0.6754 - val_loss: 0.5759 - val_auc: 0.7660
Epoch 8/40
4052/4052 [===================

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/coughvid_mfcc_26_hybrid_model_40_8_0.001/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/coughvid_mfcc_26_hybrid_model_40_8_0.001/assets


Epoch 1/40
2026/2026 [==============================] - 213s 102ms/step - loss: 0.6509 - auc: 0.6908 - val_loss: 0.5628 - val_auc: 0.7634
Epoch 2/40
2026/2026 [==============================] - 201s 99ms/step - loss: 0.6314 - auc: 0.6825 - val_loss: 0.5829 - val_auc: 0.7533
Epoch 3/40
2026/2026 [==============================] - 201s 99ms/step - loss: 0.6309 - auc: 0.6842 - val_loss: 0.5901 - val_auc: 0.7605
Epoch 4/40
2026/2026 [==============================] - 201s 99ms/step - loss: 0.6318 - auc: 0.6800 - val_loss: 0.5770 - val_auc: 0.7566
Epoch 5/40
2026/2026 [==============================] - 201s 99ms/step - loss: 0.6317 - auc: 0.6791 - val_loss: 0.5728 - val_auc: 0.7636
Epoch 6/40
166/166 [==============================] - 5s 27ms/step - loss: 0.5742 - auc: 0.7533


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/coughvid_mfcc_26_hybrid_model_40_16_0.001/assets


...

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/coughvid_mfcc_26_hybrid_model_40_64_0.001/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/coughvid_mfcc_26_hybrid_model_40_64_0.001/assets


Epoch 1/40
254/254 [==============================] - 180s 683ms/step - loss: 0.7286 - auc: 0.6806 - val_loss: 0.6289 - val_auc: 0.7410
Epoch 2/40
254/254 [==============================] - 171s 672ms/step - loss: 0.6317 - auc: 0.6851 - val_loss: 0.5984 - val_auc: 0.7639
Epoch 3/40
254/254 [==============================] - 170s 668ms/step - loss: 0.6264 - auc: 0.7001 - val_loss: 0.5637 - val_auc: 0.7651
Epoch 4/40
254/254 [==============================] - 171s 672ms/step - loss: 0.6218 - auc: 0.7094 - val_loss: 0.5861 - val_auc: 0.7596
Epoch 5/40
254/254 [==============================] - 170s 670ms/step - loss: 0.6179 - auc: 0.7159 - val_loss: 0.5725 - val_auc: 0.7697
Epoch 6/40
254/254 [==============================] - 169s 667ms/step - loss: 0.6133 - auc: 0.7227 - val_loss: 0.5900 - val_auc: 0.7714
Epoch 7/40
254/254 [==============================] - 169s 665ms/step - loss: 0.6090 - auc: 0.7283 - val_loss: 0.5864 - val_auc: 0.7681
Epoch 8/40
254/254 [============================

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/coughvid_mfcc_26_hybrid_model_40_128_0.001/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/coughvid_mfcc_26_hybrid_model_40_128_0.001/assets


## Standard scaler model

In [32]:
X_train_2d.shape

(22410, 234, 236)

In [33]:
from sklearn.preprocessing import StandardScaler


sc = StandardScaler()

num_instances, first_dim, second_dim = X_train_2d.shape
X_train_2d = np.reshape(X_train_2d, newshape=(-1, second_dim))
X_train_2d = sc.fit_transform(X_train_2d)
X_train_2d = np.reshape(X_train_2d, newshape=(num_instances, first_dim, second_dim))


num_instances, first_dim, second_dim = X_val_2d.shape
X_val_2d = np.reshape(X_val_2d, newshape=(-1, second_dim))
X_val_2d = sc.transform(X_val_2d)
X_val_2d = np.reshape(X_val_2d, newshape=(num_instances, first_dim, second_dim))


num_instances, first_dim, second_dim = X_test_2d.shape
X_test_2d = np.reshape(X_test_2d, newshape=(-1, second_dim))
X_test_2d = sc.transform(X_test_2d)
X_test_2d = np.reshape(X_test_2d, newshape=(num_instances, first_dim, second_dim))

# reshape training data

X_train_2d, X_val_2d, X_test_2d = X_train_2d[..., np.newaxis], X_val_2d[..., np.newaxis], X_test_2d[..., np.newaxis]
X_train_2d.shape

(22410, 234, 236, 1)

In [34]:
X_train_1d, X_val_1d, X_test_1d = X_train_1d[..., np.newaxis], X_val_1d[..., np.newaxis], X_test_1d[..., np.newaxis]
X_train_1d.shape

(22410, 36, 1)

In [35]:
# reshape target
import tensorflow as tf

y_train_2d = tf.one_hot(y_train_2d, depth=2)
y_test_2d = tf.one_hot(y_test_2d, depth=2)
y_val_2d = tf.one_hot(y_val_2d, depth=2)

In [36]:
y_train_2d.shape

TensorShape([22410, 2])

In [37]:
SAVED_MODEL_PATH = os.path.join(root, 'saved_model')
EPOCHS = 40
PATIENCE = 5
LEARNING_RATE = 0.001

In [38]:
from tensorflow.keras.layers import Conv1D, Conv2D, MaxPooling1D, MaxPooling2D, Concatenate
from datetime import datetime

from tensorflow.keras.layers import Input, DepthwiseConv2D
from tensorflow.keras.layers import Conv2D, BatchNormalization
from tensorflow.keras.layers import ReLU, Activation, AvgPool2D, Flatten, Dense
from tensorflow.keras.layers import Dropout, LSTM, GRU, Bidirectional

from tensorflow.keras import Model



# MobileNet block

def mobilnet_block (x, filters, strides):
    
    x = DepthwiseConv2D(kernel_size = 3, strides = strides, padding = 'same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    x = Conv2D(filters = filters, kernel_size = 1, strides = 1)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    return x

def hybrid_model(input_1, input_2):

  input_1 = Input(input_1[1:])

  x = Conv2D(filters = 32, kernel_size = 1, strides = 2, padding = 'same')(input_1)
  x = BatchNormalization()(x)
  x = ReLU()(x)

  # main part of the model

  x = mobilnet_block(x, filters = 64, strides = 1)
  x = mobilnet_block(x, filters = 128, strides = 2)
  x = mobilnet_block(x, filters = 128, strides = 1)
  x = mobilnet_block(x, filters = 256, strides = 2)
  x = mobilnet_block(x, filters = 256, strides = 1)
  x = mobilnet_block(x, filters = 512, strides = 2)

  for _ in range (5):
      x = mobilnet_block(x, filters = 512, strides = 1)
  x = mobilnet_block(x, filters = 1024, strides = 2)
  x = mobilnet_block(x, filters = 1024, strides = 1)
  x = AvgPool2D (pool_size = 7, strides = 1, data_format='channels_first')(x)

  x = Flatten()(x)
  x = Dense(units=500, activation='relu')(x)
  x = Dense(units=256, activation='relu')(x)
  x = Dense(units=128, activation='relu')(x)
  x = Dense(units=64, activation='relu')(x)
  # add meta
  # input_2 = Input((7380, 128000))
  input_2 = Input(input_2[1:])

  y = Conv1D(filters=64, kernel_size=(3), strides=1, padding='same')(input_2)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4)(y)
  y = Dropout(0.5)(y)

  # LFLB2
  y = Conv1D(filters=64, kernel_size=3, strides=1, padding='same')(y)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4, padding='same')(y)
  y = Dropout(0.25)(y)

  # LFLB3
  y = Conv1D(filters=128, kernel_size=3, strides=1, padding='same')(y)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4, padding='same')(y)
  y = Dropout(0.25)(y)

  # LFLB4
  y = Conv1D(filters=256, kernel_size=3, strides=1, padding='same')(y)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4, padding='same')(y)
  y = Dropout(0.25)(y)

  y = LSTM(units = 256, return_sequences = True, name='lstm_layer')(y)
  y = Dropout(rate = 0.2)(y) 
  y = BatchNormalization()(y)
  y = Dropout(rate = 0.2)(y)
  y = Flatten()(y)
  y = Dense(64)(y)


  merged = Concatenate()([x, y])
  output = Dense(2, activation='softmax')(merged)

  merged_model = Model(inputs=[input_1, input_2], outputs=[output])

  return merged_model

input_2d = X_train_2d.shape
input_1d = X_train_1d.shape
model = hybrid_model(input_2d, input_1d)

In [39]:
opt = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)
auc = tf.keras.metrics.AUC()
loss = tf.keras.losses.BinaryCrossentropy()
# model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

In [40]:
model.compile(optimizer=opt,
              loss=loss,
              metrics=[auc])

In [42]:
fp = f'./coswara_mfcc_{n_mfcc}_hybrid_bestweight.h5'

check = tf.keras.callbacks.ModelCheckpoint(fp,
                                           monitor='val_auc',
                                           save_best_only=True,
                                           mode='max')

earlystop_callback = tf.keras.callbacks.EarlyStopping(monitor="val_auc",
                                                      min_delta=0.001,
                                                      mode='max',
                                                      patience=PATIENCE)

log_dir = "logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [43]:
for i in [8, 16, 32, 64, 128]:
  BATCH_SIZE = i

  history = model.fit([X_train_2d, X_train_1d],\
                      [y_train_2d, y_train_1d],\
                      epochs=EPOCHS,\
                      batch_size=BATCH_SIZE,\
                      validation_data=([X_val_2d, X_val_1d], [y_val_2d, y_val_1d]),\
                      callbacks=[check, earlystop_callback, tensorboard_callback],\
                      shuffle=True)
  
  model.evaluate([X_test_2d, X_test_1d], [y_test_2d, y_test_1d])

  model.save(os.path.join(SAVED_MODEL_PATH, f'std_coughvid_mfcc_{n_mfcc}_hybrid_model_{EPOCHS}_{BATCH_SIZE}_{LEARNING_RATE}'))

  del model

  model = hybrid_model(input_2d, input_1d)

  model.compile(optimizer=opt,
               loss=loss,
               metrics=[auc])

Epoch 1/40
2802/2802 [==============================] - 191s 62ms/step - loss: 0.7143 - auc: 0.5071 - val_loss: 0.6956 - val_auc: 0.5232
Epoch 2/40
2802/2802 [==============================] - 170s 61ms/step - loss: 0.6953 - auc: 0.5245 - val_loss: 0.6598 - val_auc: 0.7617
Epoch 3/40
2802/2802 [==============================] - 170s 61ms/step - loss: 0.6948 - auc: 0.5229 - val_loss: 0.7133 - val_auc: 0.4550
Epoch 4/40
2802/2802 [==============================] - 170s 61ms/step - loss: 0.6944 - auc: 0.5232 - val_loss: 0.7387 - val_auc: 0.3989
Epoch 5/40
2802/2802 [==============================] - 169s 60ms/step - loss: 0.6937 - auc: 0.5305 - val_loss: 0.6547 - val_auc: 0.7367
Epoch 6/40
2802/2802 [==============================] - 170s 61ms/step - loss: 0.6945 - auc: 0.5204 - val_loss: 0.6555 - val_auc: 0.7622
Epoch 7/40
166/166 [==============================] - 5s 28ms/step - loss: 0.6653 - auc: 0.7560


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/std_coughvid_mfcc_26_hybrid_model_40_8_0.001/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/std_coughvid_mfcc_26_hybrid_model_40_8_0.001/assets


Epoch 1/40
1401/1401 [==============================] - 150s 102ms/step - loss: 0.7378 - auc: 0.5579 - val_loss: 0.6700 - val_auc: 0.7522
Epoch 2/40
1401/1401 [==============================] - 141s 101ms/step - loss: 0.6936 - auc: 0.5264 - val_loss: 0.6494 - val_auc: 0.7268
Epoch 3/40
1401/1401 [==============================] - 141s 100ms/step - loss: 0.6931 - auc: 0.5282 - val_loss: 0.6760 - val_auc: 0.6806
Epoch 4/40
1401/1401 [==============================] - 142s 102ms/step - loss: 0.6938 - auc: 0.5247 - val_loss: 0.6468 - val_auc: 0.7625
Epoch 5/40
1401/1401 [==============================] - 140s 100ms/step - loss: 0.6937 - auc: 0.5227 - val_loss: 0.6630 - val_auc: 0.7420
Epoch 6/40
1401/1401 [==============================] - 142s 101ms/step - loss: 0.6931 - auc: 0.5302 - val_loss: 0.6640 - val_auc: 0.7681
Epoch 7/40
1401/1401 [==============================] - 141s 101ms/step - loss: 0.6936 - auc: 0.5257 - val_loss: 0.6625 - val_auc: 0.7374
Epoch 8/40
1401/1401 [============

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/std_coughvid_mfcc_26_hybrid_model_40_16_0.001/assets


...

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/std_coughvid_mfcc_26_hybrid_model_40_64_0.001/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/std_coughvid_mfcc_26_hybrid_model_40_64_0.001/assets


Epoch 1/40
176/176 [==============================] - 128s 691ms/step - loss: 0.9245 - auc: 0.5503 - val_loss: 0.7224 - val_auc: 0.3130
Epoch 2/40
176/176 [==============================] - 119s 678ms/step - loss: 0.6926 - auc: 0.5367 - val_loss: 0.6725 - val_auc: 0.7387
Epoch 3/40
176/176 [==============================] - 119s 679ms/step - loss: 0.6897 - auc: 0.5531 - val_loss: 0.6499 - val_auc: 0.7411
Epoch 4/40
176/176 [==============================] - 122s 691ms/step - loss: 0.6866 - auc: 0.5664 - val_loss: 0.5955 - val_auc: 0.7692
Epoch 5/40
176/176 [==============================] - 120s 679ms/step - loss: 0.6825 - auc: 0.5777 - val_loss: 0.6338 - val_auc: 0.7093
Epoch 6/40
176/176 [==============================] - 119s 675ms/step - loss: 0.6788 - auc: 0.5905 - val_loss: 0.6121 - val_auc: 0.7343
Epoch 7/40
176/176 [==============================] - 120s 681ms/step - loss: 0.6736 - auc: 0.6029 - val_loss: 0.6829 - val_auc: 0.5478
Epoch 8/40
176/176 [============================

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/std_coughvid_mfcc_26_hybrid_model_40_128_0.001/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/std_coughvid_mfcc_26_hybrid_model_40_128_0.001/assets


## SMOTE

In [33]:
X_train_2d.shape

(22410, 234, 236)

In [34]:
from imblearn.over_sampling import SMOTE


sm = SMOTE(random_state=42)

num_instances, first_dim, second_dim = X_train_2d.shape
X_train_2d = np.reshape(X_train_2d, newshape=(num_instances, -1))
X_train_2d, y_train_2d = sm.fit_resample(X_train_2d, y_train_2d)
X_train_2d = np.reshape(X_train_2d, newshape=(X_train_2d.shape[0], first_dim, second_dim))


num_instances, first_dim = X_train_1d.shape
X_train_1d = np.reshape(X_train_1d, newshape=(num_instances, -1))
X_train_1d, y_train_1d = sm.fit_resample(X_train_1d, y_train_1d)
X_train_1d = np.reshape(X_train_1d, newshape=(X_train_1d.shape[0], first_dim))

# num_instances, first_dim, second_dim = X_val.shape
# X_val = np.reshape(X_val, newshape=(-1, second_dim))
# X_val = sc.transform(X_val)
# X_val = np.reshape(X_val, newshape=(num_instances, first_dim, second_dim))


# num_instances, first_dim, second_dim = X_test.shape
# X_test = np.reshape(X_test, newshape=(-1, second_dim))
# X_test = sc.transform(X_test)
# X_test = np.reshape(X_test, newshape=(num_instances, first_dim, second_dim))

# reshape training data

X_train_2d, X_val_2d, X_test_2d = X_train_2d[..., np.newaxis], X_val_2d[..., np.newaxis], X_test_2d[..., np.newaxis]
X_train_2d.shape

(23786, 234, 236, 1)

In [35]:
X_train_1d, X_val_1d, X_test_1d = X_train_1d[..., np.newaxis], X_val_1d[..., np.newaxis], X_test_1d[..., np.newaxis]
X_train_1d.shape

(23786, 36, 1)

In [36]:
# reshape target
import tensorflow as tf

y_train_2d = tf.one_hot(y_train_2d, depth=2)
y_test_2d = tf.one_hot(y_test_2d, depth=2)
y_val_2d = tf.one_hot(y_val_2d, depth=2)

In [37]:
y_train_2d.shape

TensorShape([23786, 2])

In [38]:
SAVED_MODEL_PATH = os.path.join(root, 'saved_model')
EPOCHS = 40
PATIENCE = 5
LEARNING_RATE = 0.001

In [39]:
from tensorflow.keras.layers import Conv1D, Conv2D, MaxPooling1D, MaxPooling2D, Concatenate
from datetime import datetime

from tensorflow.keras.layers import Input, DepthwiseConv2D
from tensorflow.keras.layers import Conv2D, BatchNormalization
from tensorflow.keras.layers import ReLU, Activation, AvgPool2D, Flatten, Dense
from tensorflow.keras.layers import Dropout, LSTM, GRU, Bidirectional

from tensorflow.keras import Model



# MobileNet block

def mobilnet_block (x, filters, strides):
    
    x = DepthwiseConv2D(kernel_size = 3, strides = strides, padding = 'same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    x = Conv2D(filters = filters, kernel_size = 1, strides = 1)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    return x

def hybrid_model(input_1, input_2):

  input_1 = Input(input_1[1:])

  x = Conv2D(filters = 32, kernel_size = 1, strides = 2, padding = 'same')(input_1)
  x = BatchNormalization()(x)
  x = ReLU()(x)

  # main part of the model

  x = mobilnet_block(x, filters = 64, strides = 1)
  x = mobilnet_block(x, filters = 128, strides = 2)
  x = mobilnet_block(x, filters = 128, strides = 1)
  x = mobilnet_block(x, filters = 256, strides = 2)
  x = mobilnet_block(x, filters = 256, strides = 1)
  x = mobilnet_block(x, filters = 512, strides = 2)

  for _ in range (5):
      x = mobilnet_block(x, filters = 512, strides = 1)
  x = mobilnet_block(x, filters = 1024, strides = 2)
  x = mobilnet_block(x, filters = 1024, strides = 1)
  x = AvgPool2D (pool_size = 7, strides = 1, data_format='channels_first')(x)

  x = Flatten()(x)
  x = Dense(units=500, activation='relu')(x)
  x = Dense(units=256, activation='relu')(x)
  x = Dense(units=128, activation='relu')(x)
  x = Dense(units=64, activation='relu')(x)
  # add meta
  # input_2 = Input((7380, 128000))
  input_2 = Input(input_2[1:])

  y = Conv1D(filters=64, kernel_size=(3), strides=1, padding='same')(input_2)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4)(y)
  y = Dropout(0.5)(y)

  # LFLB2
  y = Conv1D(filters=64, kernel_size=3, strides=1, padding='same')(y)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4, padding='same')(y)
  y = Dropout(0.25)(y)

  # LFLB3
  y = Conv1D(filters=128, kernel_size=3, strides=1, padding='same')(y)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4, padding='same')(y)
  y = Dropout(0.25)(y)

  # LFLB4
  y = Conv1D(filters=256, kernel_size=3, strides=1, padding='same')(y)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4, padding='same')(y)
  y = Dropout(0.25)(y)

  y = LSTM(units = 256, return_sequences = True, name='lstm_layer')(y)
  y = Dropout(rate = 0.2)(y) 
  y = BatchNormalization()(y)
  y = Dropout(rate = 0.2)(y)
  y = Flatten()(y)
  y = Dense(64)(y)


  merged = Concatenate()([x, y])
  output = Dense(2, activation='softmax')(merged)

  merged_model = Model(inputs=[input_1, input_2], outputs=[output])

  return merged_model

input_2d = X_train_2d.shape
input_1d = X_train_1d.shape
model = hybrid_model(input_2d, input_1d)

In [40]:
opt = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)
auc = tf.keras.metrics.AUC()
loss = tf.keras.losses.BinaryCrossentropy()

In [41]:
model.compile(optimizer=opt,
              loss=loss,
              metrics=[auc])

In [42]:
fp = f'./coughvid_hybrid_mfcc_{n_mfcc}_bestweight.h5'

check = tf.keras.callbacks.ModelCheckpoint(fp,
                                           monitor='val_auc',
                                           save_best_only=True,
                                           mode='max')

earlystop_callback = tf.keras.callbacks.EarlyStopping(monitor="val_auc",
                                                      min_delta=0.001,
                                                      mode='max',
                                                      patience=PATIENCE)

log_dir = "logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [43]:
for i in [8, 16, 32, 64, 128]:
  BATCH_SIZE = i

  history = model.fit([X_train_2d, X_train_1d],\
                      [y_train_2d, y_train_1d],\
                      epochs=EPOCHS,\
                      batch_size=BATCH_SIZE,\
                      validation_data=([X_val_2d, X_val_1d], [y_val_2d, y_val_1d]),\
                      callbacks=[check, earlystop_callback, tensorboard_callback],\
                      shuffle=True)
  
  model.evaluate([X_test_2d, X_test_1d], [y_test_2d, y_test_1d])

  model.save(os.path.join(SAVED_MODEL_PATH, f'smote_coughvid_mfcc_{n_mfcc}_hybrid_model_{EPOCHS}_{BATCH_SIZE}_{LEARNING_RATE}'))

  del model

  model = hybrid_model(input_2d, input_1d)

  model.compile(optimizer=opt,
               loss=loss,
               metrics=[auc])

Epoch 1/40
2974/2974 [==============================] - 212s 65ms/step - loss: 0.7151 - auc: 0.5013 - val_loss: 0.6993 - val_auc: 0.4332
Epoch 2/40
2974/2974 [==============================] - 192s 65ms/step - loss: 0.6973 - auc: 0.4998 - val_loss: 0.6965 - val_auc: 0.4801
Epoch 3/40
2974/2974 [==============================] - 192s 65ms/step - loss: 0.6968 - auc: 0.4998 - val_loss: 0.6783 - val_auc: 0.6816
Epoch 4/40
2974/2974 [==============================] - 191s 64ms/step - loss: 0.6964 - auc: 0.4992 - val_loss: 0.6400 - val_auc: 0.6966
Epoch 5/40
2974/2974 [==============================] - 190s 64ms/step - loss: 0.6958 - auc: 0.5068 - val_loss: 0.6865 - val_auc: 0.6231
Epoch 6/40
2974/2974 [==============================] - 189s 64ms/step - loss: 0.6955 - auc: 0.5048 - val_loss: 0.6684 - val_auc: 0.6822
Epoch 7/40
2974/2974 [==============================] - 189s 63ms/step - loss: 0.6961 - auc: 0.4959 - val_loss: 0.6829 - val_auc: 0.7364
Epoch 8/40
2974/2974 [===================

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/smote_coughvid_mfcc_26_hybrid_model_40_8_0.001/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/smote_coughvid_mfcc_26_hybrid_model_40_8_0.001/assets


Epoch 1/40
1487/1487 [==============================] - 186s 120ms/step - loss: 0.7313 - auc: 0.4943 - val_loss: 0.6863 - val_auc: 0.5696
Epoch 2/40
1487/1487 [==============================] - 178s 119ms/step - loss: 0.6956 - auc: 0.5001 - val_loss: 0.7043 - val_auc: 0.4547
Epoch 3/40
1487/1487 [==============================] - 177s 119ms/step - loss: 0.6946 - auc: 0.5110 - val_loss: 0.7144 - val_auc: 0.3504
Epoch 4/40
1487/1487 [==============================] - 178s 119ms/step - loss: 0.6947 - auc: 0.5083 - val_loss: 0.6379 - val_auc: 0.7336
Epoch 5/40
1487/1487 [==============================] - 177s 119ms/step - loss: 0.6940 - auc: 0.5188 - val_loss: 0.7640 - val_auc: 0.2422
Epoch 6/40
1487/1487 [==============================] - 177s 119ms/step - loss: 0.6930 - auc: 0.5279 - val_loss: 0.6783 - val_auc: 0.5734
Epoch 7/40
1487/1487 [==============================] - 177s 119ms/step - loss: 0.6900 - auc: 0.5477 - val_loss: 0.7172 - val_auc: 0.4264
Epoch 8/40
1487/1487 [============

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/smote_coughvid_mfcc_26_hybrid_model_40_16_0.001/assets


...

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/smote_coughvid_mfcc_26_hybrid_model_40_64_0.001/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/smote_coughvid_mfcc_26_hybrid_model_40_64_0.001/assets


Epoch 1/40
186/186 [==============================] - 174s 891ms/step - loss: 0.8313 - auc: 0.4837 - val_loss: 0.7196 - val_auc: 0.2330
Epoch 2/40
186/186 [==============================] - 162s 870ms/step - loss: 0.6952 - auc: 0.4995 - val_loss: 0.6957 - val_auc: 0.3862
Epoch 3/40
186/186 [==============================] - 161s 866ms/step - loss: 0.6938 - auc: 0.5065 - val_loss: 0.6989 - val_auc: 0.3922
Epoch 4/40
186/186 [==============================] - 161s 864ms/step - loss: 0.6918 - auc: 0.5246 - val_loss: 0.6938 - val_auc: 0.4945
Epoch 5/40
186/186 [==============================] - 160s 859ms/step - loss: 0.6900 - auc: 0.5344 - val_loss: 0.7002 - val_auc: 0.3980
Epoch 6/40
186/186 [==============================] - 159s 857ms/step - loss: 0.6854 - auc: 0.5657 - val_loss: 0.7128 - val_auc: 0.3768
Epoch 7/40
186/186 [==============================] - 160s 858ms/step - loss: 0.6812 - auc: 0.5845 - val_loss: 0.7582 - val_auc: 0.2788
Epoch 8/40
186/186 [============================

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/smote_coughvid_mfcc_26_hybrid_model_40_128_0.001/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/smote_coughvid_mfcc_26_hybrid_model_40_128_0.001/assets


## Evaluation

In [32]:
SAVED_MODEL_PATH = os.path.join(root, 'saved_model')

### Standard scaler

In [33]:
X_train_1d, X_val_1d, X_test_1d = X_train_1d[..., np.newaxis], X_val_1d[..., np.newaxis], X_test_1d[..., np.newaxis]

In [34]:
from sklearn.preprocessing import StandardScaler


sc = StandardScaler()

num_instances, first_dim, second_dim = X_train_2d.shape
X_train_std = np.reshape(X_train_2d, newshape=(-1, second_dim))
X_train_std = sc.fit_transform(X_train_std)
X_train_std = np.reshape(X_train_std, newshape=(num_instances, first_dim, second_dim))


num_instances, first_dim, second_dim = X_val_2d.shape
X_val_std = np.reshape(X_val_2d, newshape=(-1, second_dim))
X_val_std = sc.transform(X_val_std)
X_val_std = np.reshape(X_val_std, newshape=(num_instances, first_dim, second_dim))


num_instances, first_dim, second_dim = X_test_2d.shape
X_test_std = np.reshape(X_test_2d, newshape=(-1, second_dim))
X_test_std = sc.transform(X_test_std)
X_test_std = np.reshape(X_test_std, newshape=(num_instances, first_dim, second_dim))

# reshape training data

X_train_std, X_val_std, X_test_std = X_train_std[..., np.newaxis], X_val_std[..., np.newaxis], X_test_std[..., np.newaxis]

# reshape target
import tensorflow as tf

y_train_std = tf.one_hot(y_train_2d, depth=2)
y_test_std = tf.one_hot(y_test_2d, depth=2)
y_val_std = tf.one_hot(y_val_2d, depth=2)

In [35]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'std_coughvid_mfcc_{n_mfcc}_hybrid_model_40_128_0.001'))
loaded_model.evaluate([X_test_std, X_test_1d], [y_test_std, y_test_1d], verbose=1)

166/166 [==============================] - 16s 38ms/step - loss: 0.6939 - auc: 0.5135


[0.6938592195510864, 0.5135320425033569]

In [36]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'std_coughvid_mfcc_{n_mfcc}_hybrid_model_40_64_0.001'))
loaded_model.evaluate([X_test_std, X_test_1d], [y_test_std, y_test_1d], verbose=1)

166/166 [==============================] - 7s 37ms/step - loss: 0.6616 - auc: 0.7222


[0.6615827679634094, 0.7222089767456055]

In [37]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'std_coughvid_mfcc_{n_mfcc}_hybrid_model_40_32_0.001'))
loaded_model.evaluate([X_test_std, X_test_1d], [y_test_std, y_test_1d], verbose=1)

166/166 [==============================] - 7s 37ms/step - loss: 0.6816 - auc: 0.6187


[0.681608259677887, 0.6187475323677063]

In [38]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'std_coughvid_mfcc_{n_mfcc}_hybrid_model_40_16_0.001'))
loaded_model.evaluate([X_test_std, X_test_1d], [y_test_std, y_test_1d], verbose=1)

166/166 [==============================] - 7s 38ms/step - loss: 0.6615 - auc: 0.7666


[0.6614964604377747, 0.7665632367134094]

In [39]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'std_coughvid_mfcc_{n_mfcc}_hybrid_model_40_8_0.001'))
loaded_model.evaluate([X_test_std, X_test_1d], [y_test_std, y_test_1d], verbose=1)

166/166 [==============================] - 7s 38ms/step - loss: 0.6653 - auc: 0.7560


[0.6653217673301697, 0.7560150623321533]

### Raw features

In [40]:
# reshape training data

X_train_2d, X_val_2d, X_test_2d = X_train_2d[..., np.newaxis], X_val_2d[..., np.newaxis], X_test_2d[..., np.newaxis]
X_train_2d.shape

# reshape target
import tensorflow as tf

y_train = tf.one_hot(y_train_2d, depth=2)
y_test = tf.one_hot(y_test_2d, depth=2)
y_val = tf.one_hot(y_val_2d, depth=2)

In [41]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'coughvid_mfcc_{n_mfcc}_hybrid_model_40_128_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

166/166 [==============================] - 7s 38ms/step - loss: 0.5888 - auc: 0.7591


[0.5887835621833801, 0.7590759992599487]

In [42]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'coughvid_mfcc_{n_mfcc}_hybrid_model_40_64_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

166/166 [==============================] - 7s 38ms/step - loss: 0.5833 - auc: 0.7600


[0.5833160281181335, 0.7599533200263977]

In [43]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'coughvid_mfcc_{n_mfcc}_hybrid_model_40_32_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

166/166 [==============================] - 7s 38ms/step - loss: 0.5767 - auc: 0.7639


[0.5767391324043274, 0.7639344930648804]

In [44]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'coughvid_mfcc_{n_mfcc}_hybrid_model_40_16_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

166/166 [==============================] - 7s 38ms/step - loss: 0.5742 - auc: 0.7533


[0.5742033123970032, 0.7532594203948975]

In [45]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'coughvid_mfcc_{n_mfcc}_hybrid_model_40_8_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

166/166 [==============================] - 7s 38ms/step - loss: 0.5711 - auc: 0.7515


[0.5710521936416626, 0.7514894008636475]

### SMOTE

In [46]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'smote_coughvid_mfcc_{n_mfcc}_hybrid_model_40_128_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

166/166 [==============================] - 7s 38ms/step - loss: 0.8010 - auc: 0.3561


[0.8010057210922241, 0.3560740351676941]

In [47]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'smote_coughvid_mfcc_{n_mfcc}_hybrid_model_40_64_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

166/166 [==============================] - 8s 38ms/step - loss: 0.7615 - auc: 0.3734


[0.7614628672599792, 0.3734085261821747]

In [48]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'smote_coughvid_mfcc_{n_mfcc}_hybrid_model_40_32_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

166/166 [==============================] - 7s 38ms/step - loss: 0.6943 - auc: 0.4865


[0.6942934989929199, 0.4864582419395447]

In [49]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'smote_coughvid_mfcc_{n_mfcc}_hybrid_model_40_16_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

166/166 [==============================] - 7s 38ms/step - loss: 0.7047 - auc: 0.4468


[0.7046563625335693, 0.44678717851638794]

In [50]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'smote_coughvid_mfcc_{n_mfcc}_hybrid_model_40_8_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

166/166 [==============================] - 7s 38ms/step - loss: 0.6980 - auc: 0.4220


[0.6980229020118713, 0.42198124527931213]